# Neural Network

**Learning Objectives:**
  * Use the `DNNRegressor` class in TensorFlow to predict median housing price

The data is based on 1990 census data from California. This data is at the city block level, so these features reflect the total number of rooms in that block, or the total number of people who live on that block, respectively.
<p>
Let's use a set of features to predict house value.

## Set Up
In this first cell, we'll load the necessary libraries.

In [1]:
import math
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Next, we'll load our data set.

In [2]:
df = pd.read_csv("https://storage.googleapis.com/ml_universities/california_housing_train.csv", sep=",")

## Examine the data

It's a good idea to get to know your data a little bit before you work with it.

We'll print out a quick summary of a few useful statistics on each column.

This will include things like mean, standard deviation, max, min, and various quantiles.

In [3]:
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.3,34.2,15.0,5612.0,1283.0,1015.0,472.0,1.5,66900.0
1,-114.5,34.4,19.0,7650.0,1901.0,1129.0,463.0,1.8,80100.0
2,-114.6,33.7,17.0,720.0,174.0,333.0,117.0,1.7,85700.0
3,-114.6,33.6,14.0,1501.0,337.0,515.0,226.0,3.2,73400.0
4,-114.6,33.6,20.0,1454.0,326.0,624.0,262.0,1.9,65500.0


In [4]:
df.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0
mean,-119.6,35.6,28.6,2643.7,539.4,1429.6,501.2,3.9,207300.9
std,2.0,2.1,12.6,2179.9,421.5,1147.9,384.5,1.9,115983.8
min,-124.3,32.5,1.0,2.0,1.0,3.0,1.0,0.5,14999.0
25%,-121.8,33.9,18.0,1462.0,297.0,790.0,282.0,2.6,119400.0
50%,-118.5,34.2,29.0,2127.0,434.0,1167.0,409.0,3.5,180400.0
75%,-118.0,37.7,37.0,3151.2,648.2,1721.0,605.2,4.8,265000.0
max,-114.3,42.0,52.0,37937.0,6445.0,35682.0,6082.0,15.0,500001.0


This data is at the city block level, so these features reflect the total number of rooms in that block, or the total number of people who live on that block, respectively.  Let's create a different, more appropriate feature.  Because we are predicing the price of a single house, we should try to make all our features correspond to a single house as well

In [5]:
df['num_rooms'] = df['total_rooms'] / df['households']
df['num_bedrooms'] = df['total_bedrooms'] / df['households']
df['persons_per_house'] = df['population'] / df['households']
df.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,num_rooms,num_bedrooms,persons_per_house
count,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0
mean,-119.6,35.6,28.6,2643.7,539.4,1429.6,501.2,3.9,207300.9,5.4,1.1,3.0
std,2.0,2.1,12.6,2179.9,421.5,1147.9,384.5,1.9,115983.8,2.5,0.5,4.0
min,-124.3,32.5,1.0,2.0,1.0,3.0,1.0,0.5,14999.0,0.8,0.3,0.7
25%,-121.8,33.9,18.0,1462.0,297.0,790.0,282.0,2.6,119400.0,4.4,1.0,2.4
50%,-118.5,34.2,29.0,2127.0,434.0,1167.0,409.0,3.5,180400.0,5.2,1.0,2.8
75%,-118.0,37.7,37.0,3151.2,648.2,1721.0,605.2,4.8,265000.0,6.1,1.1,3.3
max,-114.3,42.0,52.0,37937.0,6445.0,35682.0,6082.0,15.0,500001.0,141.9,34.1,502.5


In [6]:
df.drop(['total_rooms', 'total_bedrooms', 'population', 'households'], axis = 1, inplace = True)
df.describe()

,longitude,latitude,housing_median_age,median_income,median_house_value,num_rooms,num_bedrooms,persons_per_house
count,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0
mean,-119.6,35.6,28.6,3.9,207300.9,5.4,1.1,3.0
std,2.0,2.1,12.6,1.9,115983.8,2.5,0.5,4.0
min,-124.3,32.5,1.0,0.5,14999.0,0.8,0.3,0.7
25%,-121.8,33.9,18.0,2.6,119400.0,4.4,1.0,2.4
50%,-118.5,34.2,29.0,3.5,180400.0,5.2,1.0,2.8
75%,-118.0,37.7,37.0,4.8,265000.0,6.1,1.1,3.3
max,-114.3,42.0,52.0,15.0,500001.0,141.9,34.1,502.5


## Build a neural network model

In this exercise, we'll be trying to predict `median_house_value`. It will be our label (sometimes also called a target). We'll use the remaining columns as our input features.

To train our model, we'll first use the [LinearRegressor](https://www.tensorflow.org/api_docs/python/tf/contrib/learn/LinearRegressor) interface. Then, we'll change to DNNRegressor


In [7]:
featcols = {
  colname : tf.feature_column.numeric_column(colname) \
    for colname in 'housing_median_age,median_income,num_rooms,num_bedrooms,persons_per_house'.split(',')
}
# Bucketize lat, lon so it's not so high-res; California is mostly N-S, so more lats than lons
featcols['longitude'] = tf.feature_column.bucketized_column(tf.feature_column.numeric_column('longitude'),
                                                   np.linspace(-124.3, -114.3, 5).tolist())
featcols['latitude'] = tf.feature_column.bucketized_column(tf.feature_column.numeric_column('latitude'),
                                                  np.linspace(32.5, 42, 10).tolist())

In [8]:
featcols.keys()

dict_keys(['median_income', 'persons_per_house', 'longitude', 'num_rooms', 'num_bedrooms', 'housing_median_age', 'latitude'])

In [9]:
# Split into train and eval
msk = np.random.rand(len(df)) < 0.8
traindf = df[msk]
evaldf = df[~msk]

SCALE = 100000
BATCH_SIZE= 100
OUTDIR = './housing_trained'
train_input_fn = tf.estimator.inputs.pandas_input_fn(x = traindf[list(featcols.keys())],
                                                    y = traindf["median_house_value"] / SCALE,
                                                    num_epochs = None,
                                                    batch_size = BATCH_SIZE,
                                                    shuffle = True)
eval_input_fn = tf.estimator.inputs.pandas_input_fn(x = evaldf[list(featcols.keys())],
                                                    y = evaldf["median_house_value"] / SCALE,  # note the scaling
                                                    num_epochs = 1, 
                                                    batch_size = len(evaldf), 
                                                    shuffle=False)

In [10]:
# Linear Regressor
def train_and_evaluate(output_dir, num_train_steps):
  myopt = tf.train.FtrlOptimizer(learning_rate = 0.01) # note the learning rate
  estimator = tf.estimator.LinearRegressor(
                       model_dir = output_dir, 
                       feature_columns = featcols.values(),
                       optimizer = myopt)
  
  #Add rmse evaluation metric
  def rmse(labels, predictions):
    pred_values = tf.cast(predictions['predictions'],tf.float64)
    return {'rmse': tf.metrics.root_mean_squared_error(labels*SCALE, pred_values*SCALE)}
  estimator = tf.contrib.estimator.add_metrics(estimator,rmse)
  
  train_spec=tf.estimator.TrainSpec(
                       input_fn = train_input_fn,
                       max_steps = num_train_steps)
  eval_spec=tf.estimator.EvalSpec(
                       input_fn = eval_input_fn,
                       steps = None,
                       start_delay_secs = 1, # start evaluating after N seconds
                       throttle_secs = 10,  # evaluate every N seconds
                       )
  tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

# Run training    
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
train_and_evaluate(OUTDIR, num_train_steps = (100 * len(traindf)) / BATCH_SIZE) 

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7efbcd6be3c8>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_train_distribute': None, '_task_type': 'worker', '_log_step_count_steps': 100, '_global_id_in_cluster': 0, '_service': None, '_num_ps_replicas': 0, '_is_chief': True, '_session_config': None, '_save_checkpoints_secs': 600, '_task_id': 0, '_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_tf_random_seed': None, '_model_dir': './housing_trained', '_master': '', '_keep_checkpoint_max': 5, '_num_worker_replicas': 1}
INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7efbcd6be320>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_train_distribute': None, '_task_type': 'worker', '_session_config': None, '_global_id_in_cluster': 0, '_service': None, '_save_summary_steps': 100, '_is

INFO:tensorflow:step = 1638, loss = 50.306885
INFO:tensorflow:global_step/sec: 43.2424
INFO:tensorflow:step = 1738, loss = 33.543957 (2.326 sec)
INFO:tensorflow:global_step/sec: 81.4417
INFO:tensorflow:step = 1838, loss = 38.810448 (1.230 sec)
INFO:tensorflow:Saving checkpoints for 1909 into ./housing_trained/model.ckpt.
INFO:tensorflow:Loss for final step: 91.90565.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-29-15:05:21
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./housing_trained/model.ckpt-1909
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-05-29-15:05:23
INFO:tensorflow:Saving dict for global step 1909: average_loss = 0.5966593, global_step = 1909, loss = 2023.2716, rmse = 77243.72
INFO:tensorflow:Calling model_fn.
INFO:t

INFO:tensorflow:step = 6422, loss = 46.60589 (0.429 sec)
INFO:tensorflow:global_step/sec: 257.137
INFO:tensorflow:step = 6522, loss = 38.567024 (0.393 sec)
INFO:tensorflow:global_step/sec: 236.107
INFO:tensorflow:step = 6622, loss = 68.44139 (0.420 sec)
INFO:tensorflow:global_step/sec: 276.02
INFO:tensorflow:step = 6722, loss = 51.69354 (0.362 sec)
INFO:tensorflow:global_step/sec: 269.504
INFO:tensorflow:step = 6822, loss = 48.446495 (0.371 sec)
INFO:tensorflow:global_step/sec: 256.944
INFO:tensorflow:step = 6922, loss = 37.53728 (0.390 sec)
INFO:tensorflow:global_step/sec: 284.717
INFO:tensorflow:step = 7022, loss = 137.98022 (0.352 sec)
INFO:tensorflow:Saving checkpoints for 7026 into ./housing_trained/model.ckpt.
INFO:tensorflow:Loss for final step: 123.49478.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-29-15:06:06
INFO:tensorflow:Graph

INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 11280 into ./housing_trained/model.ckpt.
INFO:tensorflow:step = 11280, loss = 106.147865
INFO:tensorflow:global_step/sec: 172.158
INFO:tensorflow:step = 11380, loss = 61.843037 (0.588 sec)
INFO:tensorflow:global_step/sec: 244.133
INFO:tensorflow:step = 11480, loss = 34.623123 (0.410 sec)
INFO:tensorflow:global_step/sec: 252.939
INFO:tensorflow:step = 11580, loss = 28.994713 (0.393 sec)
INFO:tensorflow:global_step/sec: 229.933
INFO:tensorflow:step = 11680, loss = 100.24269 (0.435 sec)
INFO:tensorflow:global_step/sec: 286.635
INFO:tensorflow:step = 11780, loss = 63.714626 (0.349 sec)
INFO:tensorflow:global_step/sec: 259.792
INFO:tensorflow:step = 11880, loss = 89.515816 (0.384 sec)
INFO:tensorflow:global_step/sec: 278.783
INFO:tensorflow:step = 11980, loss = 36.204563 (0.358 sec)
INFO:tensorflow:global_step/sec: 277.107
INFO:tensorflow:step = 12080, loss = 53.917007 (0.361 sec)
INFO:tensorflow:global_step/

In [11]:
# DNN Regressor
def train_and_evaluate(output_dir, num_train_steps):
  myopt = tf.train.FtrlOptimizer(learning_rate = 0.01) # note the learning rate
  estimator = tf.estimator.DNNRegressor(model_dir = output_dir,
                                hidden_units = [100, 50, 20],
                                feature_columns = featcols.values(),
                                optimizer = myopt,
                                dropout = 0.1)
  
  #Add rmse evaluation metric
  def rmse(labels, predictions):
    pred_values = tf.cast(predictions['predictions'],tf.float64)
    return {'rmse': tf.metrics.root_mean_squared_error(labels*SCALE, pred_values*SCALE)}
  estimator = tf.contrib.estimator.add_metrics(estimator,rmse)
  
  train_spec=tf.estimator.TrainSpec(
                       input_fn = train_input_fn,
                       max_steps = num_train_steps)
  eval_spec=tf.estimator.EvalSpec(
                       input_fn = eval_input_fn,
                       steps = None,
                       start_delay_secs = 1, # start evaluating after N seconds
                       throttle_secs = 10,  # evaluate every N seconds
                       )
  tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

# Run training    
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
tf.summary.FileWriterCache.clear() # ensure filewriter cache is clear for TensorBoard events file
train_and_evaluate(OUTDIR, num_train_steps = (100 * len(traindf)) / BATCH_SIZE) 

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7efbc8950a20>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_train_distribute': None, '_task_type': 'worker', '_log_step_count_steps': 100, '_global_id_in_cluster': 0, '_service': None, '_num_ps_replicas': 0, '_is_chief': True, '_session_config': None, '_save_checkpoints_secs': 600, '_task_id': 0, '_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_tf_random_seed': None, '_model_dir': './housing_trained', '_master': '', '_keep_checkpoint_max': 5, '_num_worker_replicas': 1}
INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7efbc24d3b70>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_train_distribute': None, '_task_type': 'worker', '_session_config': None, '_global_id_in_cluster': 0, '_service': None, '_save_summary_steps': 100, '_is

INFO:tensorflow:step = 4107, loss = 86.81098
INFO:tensorflow:global_step/sec: 178.147
INFO:tensorflow:step = 4207, loss = 38.645973 (0.566 sec)
INFO:tensorflow:global_step/sec: 227.187
INFO:tensorflow:step = 4307, loss = 70.40639 (0.440 sec)
INFO:tensorflow:global_step/sec: 234.315
INFO:tensorflow:step = 4407, loss = 48.397625 (0.427 sec)
INFO:tensorflow:global_step/sec: 214.029
INFO:tensorflow:step = 4507, loss = 51.290554 (0.468 sec)
INFO:tensorflow:global_step/sec: 226.773
INFO:tensorflow:step = 4607, loss = 31.26803 (0.440 sec)
INFO:tensorflow:global_step/sec: 214.874
INFO:tensorflow:step = 4707, loss = 56.060295 (0.466 sec)
INFO:tensorflow:global_step/sec: 213.491
INFO:tensorflow:step = 4807, loss = 41.932533 (0.468 sec)
INFO:tensorflow:global_step/sec: 231.215
INFO:tensorflow:step = 4907, loss = 69.466774 (0.435 sec)
INFO:tensorflow:global_step/sec: 215.12
INFO:tensorflow:step = 5007, loss = 25.564297 (0.462 sec)
INFO:tensorflow:global_step/sec: 238.777
INFO:tensorflow:step = 510

INFO:tensorflow:global_step/sec: 220.257
INFO:tensorflow:step = 10066, loss = 23.966297 (0.453 sec)
INFO:tensorflow:global_step/sec: 244.322
INFO:tensorflow:step = 10166, loss = 54.62225 (0.408 sec)
INFO:tensorflow:Saving checkpoints for 10208 into ./housing_trained/model.ckpt.
INFO:tensorflow:Loss for final step: 41.064728.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-29-15:08:23
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./housing_trained/model.ckpt-10208
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-05-29-15:08:23
INFO:tensorflow:Saving dict for global step 10208: average_loss = 0.38807273, global_step = 10208, loss = 1315.9546, rmse = 62295.48
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensor

In [13]:
from google.datalab.ml import TensorBoard
pid = TensorBoard().start(OUTDIR)

TensorBoard was started successfully with pid 9159. Click here to access it.

In [14]:
TensorBoard().stop(pid)